In [1]:
import glob
import re
import pandas as pd
from collections import defaultdict, Counter
import json
import shutil, os

In [2]:
def print_au_articles(articles):
    strings = []
    for a in articles:
        title = a['title']
        if title[-1] in ['.', '!', '?', '…']:
            point = ''
        else:
            point = '.'
            
        if 'fn' in a.keys():
            s = f"- [{title}](files/{a['fn']}){point} {a['year']}. № {a['number']}."
            print(s)
        else:
            s = f"- {a['title']}{point} {a['year']}. № {a['number']}."
            print(s)
        strings.append(s)
    return strings

In [3]:
def txt2md(s, title=None, font_style=None):
    modified_s = s
    if title:
        modified_s = f"{''.join(['#' for i in range(title)])} {s}"
    if font_style == 'b':
        modified_s = f"**{s}**"
    elif font_style == 'i':
        modified_s = f"*{s}*"
    return modified_s
    

In [16]:
with open('all_articles.json', 'r') as f:
    all_articles = json.load(f)

In [17]:
len(all_articles)

1082

In [20]:
for k, v in all_articles.copy().items():
    if v['year'] == 2019:
        all_articles.pop(k)     

In [21]:
len(all_articles)

1072

## data to md

### authors

In [22]:
persons = defaultdict(list)
for k, v in all_articles.items():
    if 'authors' in v.keys():
        authors = [i['person'] for i in v['authors'] if i['person'] != '']
        if authors:
            for a in authors:
                persons[a].append(k)     

In [23]:
persons['Божков О.'][:3]

['91', '128', '278']

In [24]:
persons['Илле М.'][:3]

['28', '33', '85']

In [25]:
authors = list(persons.keys())
authors.sort()
authors[:10]

['Абрамова Л.',
 'Алексеев А.',
 'Алексеев И.',
 'Андреева Е.',
 'Андрюшкина И.',
 'Анушкова Ю.',
 'Аргунова Е.',
 'Аронсон П.',
 'Артемов В.',
 'Асочаков Ю.']

In [26]:
all_articles['999']

{'id': 1067,
 'fk_theme': 23,
 'title': 'Образ «идеального» петербуржца и культурная жизнь города',
 'header': nan,
 'body': nan,
 'file_name': '/files/2017/2017-4-04.pdf',
 'active': 'Y',
 'topic': 'Социокультурные исследования',
 'year': 2017,
 'number': 4,
 'authors': [{'author': 'М. Илле', 'for_del': 0, 'person': 'Илле М.'},
  {'author': 'СПб Университет гражданской авиации',
   'for_del': 0,
   'person': ''},
  {'author': '«Телескоп»: журнал социологических маркетинговых исследований',
   'for_del': 0,
   'person': ''}],
 'fn': '2017-4-04.pdf'}

In [27]:
txt2md('d', title=2)

'## d'

In [28]:
# ### А

# **Андреев**

# - ст.1
# - ст.2

In [29]:
page_txt = []

for i in range(ord('А'), ord('Я')):
    inner_dct = dict()
    txt = []
    letter = chr(i)
    letter_s = f"\n{txt2md(letter, title=3)}\n"
    print(letter_s)
    
    for a in authors:
        if a[0] == letter:
            author_s = f"\n{txt2md(a, font_style='b')} ({len(persons[a])})\n"
            print(author_s)
            txt.append(author_s)
            curr_au_articles = [all_articles[i] for i in all_articles if i in persons[a]]
            curr_au_articles_sorted = sorted(curr_au_articles, key=lambda d: (d['year'], d['number']), reverse=True)
            au_ar = print_au_articles(curr_au_articles_sorted)
            txt.append('\n'.join(au_ar))
    if txt:
        txt = [letter_s] + txt
        inner_dct[letter] = txt
    if inner_dct:
        page_txt.append(inner_dct)
            


### А


**Абрамова Л.** (1)

- [Изучение эффективности деятельности аппарата КПСС. Мало известные страницы истории ленинградской социологии](files/article_content140481096774373file.pdf). 2013. № 4.

**Алексеев А.** (27)

- [А.Н. Алексеев. Мои пожелания и/или распоряжения](files/2017-6-07.pdf). 2017. № 6.
- [Моя жилищная история. Опыт тематически фокусированного автобиографического повествования](files/2016-5-05.pdf). 2016. № 5.
- [О мере легитимности Правителя](files/article_content1478173144142810file.pdf). 2015. № 6.
- [Историко-биографические поиски и открытия Бориса Докторова](files/article_content1476364857275833file.pdf). 2015. № 1.
- [Созидатель. По случаю 85-летия Бориса Максимовича Фирсова](files/article_content144473435137663file.pdf). 2014. № 3.
- [«Телескоп» — на марше](files/article_content1404811102113417file.pdf). 2013. № 4.
- [Многообразие экспликаций и синонимический ряд "наблюдающего   участия" (Опыт текстологического анализа, с тяготением к историко-научному)](file

In [30]:
page_txt[-1]

{'Ю': ['\n### Ю\n',
  '\n**Юран Д.** (1)\n',
  '- [«Острова в океане»: проблемы массовой коммуникации в социальной теории](files/article_content1380212261138832file.pdf). 2012. № 3.',
  '\n**Юхнева Н.** (1)\n',
  '- [Письмо по поводу рецензии на книгу "Многонациональный Петербург. История. Религии. Народы"](files/article_content1198483612373361file.pdf). 2004. № 4.']}

In [31]:
a_dct = {
    'meta': ['---\ntitle: "Авторы"\n---\n\n## Список статей по авторам\n']   
}

In [32]:
page_txt.insert(0, a_dct)

In [33]:
with open('old_telescope/authors.qmd', 'w') as f:
    for i in page_txt:
        for k, v in i.items():
            for s in v:
                f.writelines(s + '\n')      

In [34]:
len(persons)

375

### issues

In [35]:
all_articles['999']

{'id': 1067,
 'fk_theme': 23,
 'title': 'Образ «идеального» петербуржца и культурная жизнь города',
 'header': nan,
 'body': nan,
 'file_name': '/files/2017/2017-4-04.pdf',
 'active': 'Y',
 'topic': 'Социокультурные исследования',
 'year': 2017,
 'number': 4,
 'authors': [{'author': 'М. Илле', 'for_del': 0, 'person': 'Илле М.'},
  {'author': 'СПб Университет гражданской авиации',
   'for_del': 0,
   'person': ''},
  {'author': '«Телескоп»: журнал социологических маркетинговых исследований',
   'for_del': 0,
   'person': ''}],
 'fn': '2017-4-04.pdf'}

In [36]:
[(k, v) for k, v in all_articles.items() if 'year' not in v.keys()]

[]

In [37]:
set([v['year'] for k, v in all_articles.items()])

{1997,
 1998,
 1999,
 2000,
 2001,
 2002,
 2003,
 2004,
 2005,
 2006,
 2007,
 2008,
 2009,
 2010,
 2011,
 2012,
 2013,
 2014,
 2015,
 2016,
 2017,
 2018}

In [38]:
[(k) for k, v in all_articles.items() if not isinstance(v['year'], int)]

[]

In [39]:
[(k) for k, v in all_articles.items() if not isinstance(v['number'], int)]

[]

In [40]:
[(k) for k, v in all_articles.items() if not isinstance(v['id'], int)]

[]

In [41]:
articles = [v for k, v in all_articles.items()]

In [42]:
sorted_articles = sorted(articles, key=lambda d: (d['year'], d['number'], -d['id']), reverse=True)

In [43]:
sorted_articles[-1]

{'id': 180,
 'fk_theme': 22,
 'title': 'Петербургские жители и их доходы',
 'header': nan,
 'body': nan,
 'file_name': nan,
 'active': 'Y',
 'topic': 'Социальные проблемы',
 'year': 1997,
 'number': 1,
 'authors': [{'author': 'Исследовательская фирма «Крона Корсинто»',
   'for_del': 0,
   'person': ''}]}

In [44]:
page_txt = []

curr_year = ''
curr_number = ''

for i in sorted_articles:
#     print(i)
    if i['year'] != curr_year:
        year_s = f"\n{txt2md(i['year'], title=2)}"
        print(year_s)
        page_txt.append(year_s)
    curr_year = i['year']
    if i['number'] != curr_number:
        curr_number = f"\n{txt2md(i['number'], title=3)}\n"
        print(curr_number)
        page_txt.append(curr_number)
    curr_number = i['number']
    if 'authors' in i.keys():
        authors = [a['person'] for a in i['authors'] if a['person'] != '']
        authors = ', '.join(authors)
    else:
        authors = ''
    if 'fn' in i.keys():
        art_s = f"- {authors} [{i['title']}](files/{i['fn']}) ".strip()
    else:
        art_s = f"- {authors} {i['title']}".strip()
    print(art_s)
    art_s = re.sub(r'\s+', ' ', art_s)
    page_txt.append(art_s)
    
  
            
            


## 2018

### 6

- Докторов Б. [Нариман Аитов. Основатель социологии в Башкортостане](files/2018-6-01.pdf)
- Сафронов В. [Голосование за радикальные правые партии в Европе: роль культурного изменения и партийной поляризации (Часть I)](files/2018-6-02.pdf)
- Доровская Н. [Психологические трудности в работе интервьюеров социологических колл-центров (cati) и способы их преодоления](files/2018-6-03.pdf)
- Гуревич Ю. [Как мы находим дорогу?](files/2018-6-04.pdf)
- Божков О. [33 экспедиционных дня 2018 года (заметки с социологического поля)](files/2018-6-05.pdf)
- Моор В. [Поэтические зарисовки к «афоризмам житейской мудрости» Артура Шопенгауэра](files/2018-6-06.pdf)
-  [Содержание 1-6 номеров журнала за 2018 год](files/2018-6-07.pdf)

### 5

-  [Роману Семеновичу Могилевскому — 80 лет!](files/2018-5-01.pdf)
-  [Л.А. Седов: «Я родился в 1934 в Москве, на Арбате»](files/2018-5-02.pdf)
- Гофман А. [Несколько слов о Леониде Александровиче Седове](files/2018-5-03.pdf)
- Гилинский Я. [О главных п

In [45]:
page_txt[-2:]

['- Илле М. Должен ли Б. Ельцин уйти в отставку?',
 '- Петербургские жители и их доходы']

In [46]:
meta = '---\ntitle: "Выпуски"\n---\n\nНа сайте размещены статьи журнала, опубликованные в 1997 - 2018 гг.'

In [47]:
page_txt.insert(0, meta)

In [48]:
with open('old_telescope/index.qmd', 'w') as f:
    for i in page_txt:
        f.writelines(i + '\n')      

## sections

In [49]:
all_articles['999']

{'id': 1067,
 'fk_theme': 23,
 'title': 'Образ «идеального» петербуржца и культурная жизнь города',
 'header': nan,
 'body': nan,
 'file_name': '/files/2017/2017-4-04.pdf',
 'active': 'Y',
 'topic': 'Социокультурные исследования',
 'year': 2017,
 'number': 4,
 'authors': [{'author': 'М. Илле', 'for_del': 0, 'person': 'Илле М.'},
  {'author': 'СПб Университет гражданской авиации',
   'for_del': 0,
   'person': ''},
  {'author': '«Телескоп»: журнал социологических маркетинговых исследований',
   'for_del': 0,
   'person': ''}],
 'fn': '2017-4-04.pdf'}

In [50]:
articles = [v for k, v in all_articles.items()]

In [51]:
[(k, v) for k, v in all_articles.items() if 'topic' not in v.keys()]

[]

In [52]:
len(set([v['topic'] for k, v in all_articles.items()]))

25

In [53]:
Counter([v['topic'] for k, v in all_articles.items()])

Counter({'Современная история российской социологии': 155,
         'Социальные проблемы': 124,
         'Социокультурные исследования': 87,
         'Методология и методы': 84,
         'Потребительский рынок': 77,
         'Социально-политические исследования': 75,
         'Информация, сообщения': 69,
         'Анализ рынка масс-медиа': 55,
         'Размышления': 44,
         'Компьютеры, средства связи и информационные технологии.': 40,
         'Публикации на разные темы': 37,
         'Книжная полка': 34,
         'Проблемы профессионального сообщества': 29,
         'Прошлое и настоящее в изучении общественного мнения и рекламы': 28,
         'Некролог': 25,
         'Культурная жизнь Петербурга': 20,
         'Рецензии': 20,
         'Социология литературы': 13,
         'Социология образования': 12,
         'Социология религии': 10,
         'Эссе': 10,
         'Исследования кино': 9,
         'Поведение потребителей': 6,
         'Полемические заметки': 6,
         'Социал

In [54]:
sections_order = [
    'Современная история российской социологии',
    'Размышления',
    'Методология и методы',
    'Социально-политические исследования',
    'Социальные проблемы',
    'Социокультурные исследования',
    'Культурная жизнь Петербурга',
    'Социология литературы',
    'Социология образования',
    'Социология религии',
    'Исследования кино',
    'Социально-психологические исследования',
    'Прошлое и настоящее в изучении общественного мнения и рекламы',
    'Потребительский рынок',
    'Поведение потребителей',
    'Анализ рынка масс-медиа',
    'Компьютеры, средства связи и информационные технологии.',
    'Проблемы профессионального сообщества',
    'Эссе',
    'Полемические заметки',
    'Информация, сообщения',
    'Рецензии',
    'Книжная полка',
    'Некролог',
    'Публикации на разные темы'
]


In [55]:
set([v['topic'] for k, v in all_articles.items()]).difference(sections_order)

set()

In [56]:
sorted_articles = sorted(articles, key=lambda d: (d['year'], d['number'], -d['id']), reverse=True)

In [57]:
sorted_articles[-1]

{'id': 180,
 'fk_theme': 22,
 'title': 'Петербургские жители и их доходы',
 'header': nan,
 'body': nan,
 'file_name': nan,
 'active': 'Y',
 'topic': 'Социальные проблемы',
 'year': 1997,
 'number': 1,
 'authors': [{'author': 'Исследовательская фирма «Крона Корсинто»',
   'for_del': 0,
   'person': ''}]}

In [58]:
page_txt = []

curr_section = ''
curr_year = ''
curr_number = ''

for s in sections_order:
    section_s = f"\n{txt2md(s, title=2)}\n"
    print(section_s)
    page_txt.append(section_s)   
    for sa in sorted_articles:
        if sa['topic'] == s:
            if sa['title'][-1] in ['.', '!', '?']:
                point = ''
            else:
                point = '.'
            if 'authors' in sa.keys():
                authors = [a['person'] for a in sa['authors'] if a['person'] != '']
                authors = ', '.join(authors)
            else:
                authors = ''
            if 'fn' in sa.keys():
                art_s = f"- {authors} [{sa['title']}](files/{sa['fn']}){point} {sa['year']}. №&nbsp;{sa['number']}.".strip()
            else:
                art_s = f"- {authors} {sa['title']}{point} {sa['year']}. №&nbsp;{sa['number']}.".strip()
            art_s = re.sub(r'\s+', ' ', art_s)
            print(art_s)
            page_txt.append(art_s)


## Современная история российской социологии

- Докторов Б. [Нариман Аитов. Основатель социологии в Башкортостане](files/2018-6-01.pdf). 2018. №&nbsp;6.
- [Роману Семеновичу Могилевскому — 80 лет!](files/2018-5-01.pdf) 2018. №&nbsp;5.
- [Л.А. Седов: «Я родился в 1934 в Москве, на Арбате»](files/2018-5-02.pdf). 2018. №&nbsp;5.
- Гофман А. [Несколько слов о Леониде Александровиче Седове](files/2018-5-03.pdf). 2018. №&nbsp;5.
- Докторов Б. [Уникальное историко-биографическое наследие А.Н. Алексеева](files/2018-5-09.pdf). 2018. №&nbsp;5.
- [Н.П. Попов: «Я думаю, Джордж Гэллап был бы нами доволен»](files/2018-4-01.pdf). 2018. №&nbsp;4.
- Докторов Б. [Юло Вооглайд: социолог, политик, гражданин](files/2018-4-04.pdf). 2018. №&nbsp;4.
- [Ю. А. Француз: «Я описал только малую толику всего пережитого и пройденного»](files/2018-3-01.pdf). 2018. №&nbsp;3.
- Докторов Б. [Разговоры с Валерием Голофастом: мысленные и реальные](files/2018-2-01.pdf). 2018. №&nbsp;2.
- [«1990 год. Я покинул кипящую пере

In [59]:
meta = '---\ntitle: "Рубрики"\n---\n\n'

In [60]:
page_txt.insert(0, meta)

In [61]:
with open('old_telescope/sections.qmd', 'w') as f:
    for i in page_txt:
        f.writelines(i + '\n')      

## files

In [62]:
f_pths = glob.glob('files2/*.*')
len(f_pths)

917

In [63]:
f_pths[:3]

['files2/article_content1208530395122716file.pdf',
 'files2/article_content140437849487490file.pdf',
 'files2/article_content1193062013246879file.pdf']

In [64]:
links = [v['fn'] for k, v in all_articles.items() if 'fn' in v.keys()]
len(links)

906

In [65]:
len(set(links))

906

In [66]:
f_names = [i.split('/')[-1] for i in f_pths]
len(f_names)

917

In [67]:
len(set(f_names))

917

In [68]:
len(set(links).intersection(set(f_names)))

906

In [69]:
len(set(links).difference(set(f_names)))

0

In [70]:
len(set(f_names).difference(set(links)))

11

In [71]:
for fn, fp in zip(f_names, f_pths):
    if fn in links:
        shutil.copy2(os.path.join('files2', fn), os.path.join('old_telescope/_site/files', fn))
    else:
        shutil.copy2(os.path.join('files2', fn), os.path.join('wo_links', fn))

In [59]:
# for k, v in all_articles.items():   
#     if 'в 2014 году' in v['title']:
#         print(k, v)
